<a id='top'></a>

<h1>Stream Cipher</h1>
<br>
Notes on Lecture 3, [Introduction to Cryptography](https://www.youtube.com/channel/UC1usFRN4LCMcfIV7UjHNuQg/videos), by Christof Paar, and Chapter 2, [Understanding Crpytography](https://www.amazon.com/Understanding-Cryptography-Christof-Paar-ebook/dp/B00HWUO98A), by Paar & Pletzl:<br>
<br>
Note: Many notebook functions and diagrams are unavailable /  will not display in static views (like GitHub). [View 'live' on MyBinder](https://mybinder.org/v2/gh/jinjagit/Cryptography/master?filepath=StreamCipher.ipynb)

In [ ]:
# Run this code cell for help on using 'live' notebooks
f = open('HowTo.txt', 'r'); data = f.read(); print(data)

## Contents:
<a href='#intro'>Introduction</a><br>
<a href='#rngs'>Random Number Generators</a><br>
<a href='#one_time'>The One Time Pad</a><br>
<a href='#lcg'>Linear Congruential Generator (LCG)</a><br>
<a href='#dvpa'>Decrypting the LCG Stream Cipher (via plaintext attack)</a><br>
<a href='#eea'>Extended Euclidean Algorithm and Modular Inverse</a><br>

**NOTE: The following code should be run before other code in this notebook is run** (imports necessary libraries):

In [1]:
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact,FloatSlider,IntSlider # Unused, as yet
from IPython.display import HTML as html_print

print("libraries imported")  # <- run and wait until output prints; may take a few seconds to run.

libraries imported


<a id='intro'></a>
## Introduction:

<center><img src=diagrams/CryptoTree.svg / ></center>

Note: GSM is used by all mobile phones, was developed in mid 1980s, and was first large-scale application of encrytpion. GSM uses a stream cipher to encrypt digital voice (audio) signals before transmitting to base towers.

**Definition:** A stream cipher encrypts bits individually (as against a block cipher, which encodes blocks of bits), **by adding a bit from a <i>key stream</i> to a plaintext bit.**

Usually, stream ciphers use very simple encrytption and decryption methods.

Each bit $x_i$ is encrypted by adding a secret key stream bit $s_i$, modulo 2.

<center><img src=diagrams/asynchStream.svg / ></center>

Stream ciphers tend to be small and fast and are, therefore, relevant to applications that run in environments with relatively limited resources. In general, however, block ciphers are still used more frequenly for encrytping computer communications. Neither kind can claim better overall efficiency (of software = cycles; or hardware = fewer gates/smaller chip area).

#### Encryption and Decryption:
<i>The plaintext, ciphertext and key consist of individual bits;</i>

$x_i, y_i, s_i\in\{1, 0\}$

**Encryption:** $y_i = e_{s_i}(x_i)\equiv x_i + s_i\mod{2}$<br></br>
**Decryption:** $x_i = d_{s_i}(y_i)\equiv y_i + s_i\mod{2}$

  1. Encryption and decryption are same functions!
  2. Why can we use a simple modulo 2 addition as encrytpion?
  3. What is the nature of the stream bits $s_i$?

**Proof:** that decryption uses same algorithm as encryption, by substitution of $y_i$ in decryption algorithm with the encryption algorithm, $x_i + s_i$ and further reduction of the resultant expression, thus:<br>
<br>
$d_{s_i}(y_i)\equiv y_i + s_i\mod{2}$ <br>
$d_{s_i}(y_i)\equiv (x_i + s_i) + s_i\mod{2}$ <br>
$d_{s_i}(y_i)\equiv x_i + 2s_i\mod{2}$ <br>
$d_{s_i}(y_i)\equiv x_i + 0\mod{2}$ <br>
$d_{s_i}(y_i)\equiv x_i\mod{2} \;\blacksquare$

Thus, the reason that we can use simple modulo 2 addition as decryption, (as well as encryption), is that $x_i, y_i, s_i \in\{1, 0\}$, and $x_i, y_i, s_i \in Z 2$. In other words, because they are all single bits they can only have values of either $0$, or $1$.

**RULE:** Modulo 2 addition and subtraction are the same operation.

In [ ]:
# calculates a+b mod 2, and a-b mod 2

a = int(input("enter 1st integer: "))
b = int(input("enter 2nd integer: "))

c = (a + b) % 2
d = (a - b) % 2

print("%d + %d, modulo 2 = %d" % (a, b, c))
print("%d - %d, modulo 2 = %d" % (a, b, d))

Thus, $\;\;\;\;d_{s_i}(y_i)\equiv y_i + s_i\mod{2}\;\;\;\;$ and $\;\;\;\;d_{s_i}(y_i)\equiv y_i - s_i\mod{2}\;\;\;\;$are equivalent.

Consider the truth table for $x$, $s$ and $y$:

|        |  x  |  s  |  y = x + s  |  y = x - s  |
|--------|-----|-----|-------------|-------------|
| case 1 |  0  |  0  |      0      |      0      |
| case 2 |  0  |  1  |      1      |      1      |
| case 3 |  1  |  0  |      1      |      1      |
| case 4 |  1  |  1  |      0      |      0      |

The above represents an [XOR](https://en.wikipedia.org/wiki/XOR_gate) binary gate, executed using either addition or subtraction in modulo 2.<br>
An XOR gate implements an **exclusive or**; that is, a true output results if one, and only one, of the inputs to the gate is true.

Idea: To apply modulo 2 to any binary integer value, we need only read the value of the last bit, (0 or 1), which gives the modulo 2 value of the integer. Not a great insight, as really it is another way of saying that the nature of binary means the 'units' digit of any integer will equal the modulo 2 value. This is true of the units digit for any base where a modulo of the same value as the base is applied (e.g. base 10 modulo 10). With binary, however, there is also the advantage that reading a single bit from memory is probably generally faster than doing modulo math on an integer. However, <i>I imagine</i> most decent math libraries / languages exploit this fact when performing modulo 2 math on an integer (but, I do not know this).

In [ ]:
# compare modulo 2 representation of an integer with the last digit of its binary representation

a = int(input("enter integer: "))

get_binary = lambda x: format(x, 'b') # useful function for converting integer to binary string

binary = get_binary(a)
last_digit = binary[-1:]
mod2 = a % 2

print("binary representation: " + binary)
print("last binary digit: " + last_digit)
print("%d modulo 2 = %d modulo 2" % (a, mod2))

<center><img src=diagrams/streamBasic.svg / ></center>

Considering the above truth table and its consequences, we can see that <i>if the probability of any stream bit $\mathit{s_i}$ being $\mathit{1}$ or $\mathit{0}$ is equal,</i> then the encoding of either $x_i = 0$, or $x_i = 1$, has an equal chance of giving $y_i = 0$, or $y_i = 1$, in either case.
#### Flipping bits:
Also, the value of the keystream bit has a particular effect on the plaintext $x$ value provided: $s = 1$ will always flip the input bit; $0\,\to\,1$, or $1\,\to\,0$, whereas $s = 0$ will not change the input bit.<br>
  As there are two modulo 2 addition (XOR) operations in the encoding/decoding stream, each using $s_i$ to enccode $x_i$ and decode $y_i$, $x_i$ is eventually either flipped twice; if $s_i =1$, or not at all; if $s_i =0$.

In [6]:
# XOR on 4 cases of x + s, applied twice to emulate encryption and decryption of single bits.
# Note the replacement of modular arithmetic with a single XOR operation.

def get_XOR(x, s):
    if x == s:
        return 0

    return 1

a = str(get_XOR(0,0))  
b = str(get_XOR(1,0)) 
c = str(get_XOR(0,1)) 
d = str(get_XOR(1,1)) 

def col_str(s):  # use html to print color output (white text on blue background)
    return "<text style=color:white;background-color:blue>{}</text>".format(s)

def bl_sp(s):  # use html to print blue space (blue text on blue background)
    return "<text style=color:blue;background-color:blue>{}</text>".format(s)

html_print("encrypt " + bl_sp('.') + col_str('0') + bl_sp('.') + " with s = 0 -> " + a +  
           " ... decrypt " + a + " with s = 0 -> " + bl_sp('.') + col_str(a) + bl_sp('.') + "</br>"
           + "encrypt " + bl_sp('.') + col_str('1') + bl_sp('.') + " with s = 0 -> " + b +
           " ... decrypt " + b + " with s = 0 -> " + bl_sp('.') + col_str(b) + bl_sp('.') + "</br>"
           + "encrypt " + bl_sp('.') + col_str('0') + bl_sp('.') + " with s = 1 -> " + c +
           " ... decrypt " + c + " with s = 1 -> " + bl_sp('.') + col_str(d) + bl_sp('.') + "</br>"
           + "encrypt " + bl_sp('.') + col_str('1') + bl_sp('.') + " with s = 1 -> " + d +
           " ... decrypt " + d + " with s = 1 -> " + bl_sp('.') + col_str(c) + bl_sp('.'))               

Example: Encoding the ASCII character "A":

It's number 65 in the ASCII table, which is 1000001 in binary.
We apply a pre-defined stream of 0101101:

$x_7...x_1$ $= 1000001$ <br>
$s_7...s_1$ $= 0101101$

In [ ]:
# Encrypting an ASCII 'A' in binary (requires previous code cell to have run)

def encr_bin_str(data_in, key_stream): # N.B. Since encryption only uses XOR, the same function also decrypts
    encoded = ""
    for i in range(len(data_in) * -1, 0):
        encoded = encoded + str(get_XOR(data_in[i], key_stream[i]))
        
    return encoded

data_in = "1000001"
key_stream = "0101101"

encoded = encr_bin_str(data_in, key_stream)
print("data in = %s, after encryption = %s" % (data_in, encoded))
decoded = encr_bin_str(encoded, key_stream)  # note use of encryption algo to decrypt
print("encoded = %s, after decryption = %s" % (encoded, decoded))

<a id='rngs'></a>
**Fundamental issue:** The generation of key stream bits needs to be _somehow_ related to randomness.<br>
<h2>Random Number Generators (RNG):</h2><br>
Three main types:<br>
  1. #### True Random Number Generator (TRNG):
  True random numbers stem from randomness in some physical processes; E.g. Flipping a coin, roulette, noise (thermal), computer mouse movement, keystroke timing, radioactive decay, etc. The lack of repeatability of such randomness makes TRNG problematic for key generation (including of key streams).
  2. #### Pseudo-random Number Generator (PRNG):
  PRNs are computed and, therefore, deterministic (not random). From certain points of view, however, they may _appear_ random (unpredictable). Generally, they are not adequate for cryptographic use, despite being useful in many other areas of computing (e.g. some software testing). In practice, the output from many common PRNGs exhibit artifacts that cause them to fail statistical pattern-detection tests [[Wikipedia]](https://en.wikipedia.org/wiki/Pseudorandom_number_generator). They  often calculate PRNs using the formula; $s_0 =$ seed, $s_{i+1} = f(s_i)$, recursively repeated.
  3. #### Cryptographically Secure PRNG (CPRNG):
  Also computed and uses an update function (recursion), like a PRNG, but additonally have the property that the numbers they produce <i>are</i> unpredictable. Specifically, this means that given $n$ consecutive bits of the key stream, there is no [polynomial time](http://mathworld.wolfram.com/PolynomialTime.html) algorithm that can predict the next bit $s_{n+1}$ with better than 50% chance of success. Informally, this definition is reduced to: it is computationally infeasible to compute $s_{n+1}$.

In [ ]:
# Emulating rand() function in ANSI C, a linear congruential generator (LCG) generator (a type of PRNG). Fails if seed = 0.
# Based on code from: http://pubs.opengroup.org/onlinepubs/009695399/functions/rand.html

x = int(input("enter a seed value (an integer): "))
keystream = list(range(10))
binary = list(range(10))

for i in range (0, 10):
    x = int((x * 1103515245 + 12345) % (2**31))
    keystream[i] = x
    binary[i] = get_binary(x)
    
print("\npseudo-random sequence of 10 integers: \n", end='')
print(keystream)
print("\nsequence in binary: \n", end='')
print(binary)

l = len(get_binary(2**31))
print("\nmodulo (2^31) gives a maximum length of each value as %d bits." % l)

<a id='one_time'></a>

<h2>The One Time Pad (OTP):</h2><br>
Goal: to build the 'perfect' cipher.<br>
<br>
**Definition:** a cipher is **'unconditionally secure'** (or information theoretically secure) if it cannot be broken even with <i>**infinite**</i> computing resources.<br>
<br>
The OTP is a stream cipher where:
  1. The key stream bits stem from a TRNG.
  2. Each key stream bit is used only once.
  
Of course, the problem now resides in how to communicate the key to the receiver, since the key is as long as the message!  E.g. encrypting a 400mb video file produces 3.8Gb of key. If the key is safely communicated, the key still needs to be kept safe, and it is potentially very large. Also, if the key is safely communicated, then, given the key is as big as the original data, then simply communicatingg the original message using the same method considered viable for the key should be considered. Not only this, but the key also is non-reusable (one time) if security is to be maintained.<br>
<br>
Thus, we can see OTPs may be useful in situations where the key can be communicated before the communication becomes threatened. For example, if a spy, or an aly, can carry a stash of pre-defined keys into an espionage situation before communications are likely to be intercepted, and can keep the keys safe / destroy them when used, this system may be very secure. For remote, network based encrytption, however, these stipulations normally cannot be met.
Using the Linear Congruential Generator (LCG), which uses a keystream $s_i$ from a PRNG, looks more promising (but, eventually, will be shown to be inadequate):

<a id='lcg'></a>
### Linear Congruential Generator (LCG):
- Example of simple PRNG (ANSII C rand(), which is also an LCG):<br>
<br>
$S_0 = A\cdot S_i+B, \mod{m}$, where $A, B, S_i \in Z m$<br>
Key $K = (A, B)$<br>
$A,B,S_i$ are $[log_2 m]$ bits long<br>
<br>
Note use of large ' $S$ ' to denote multi-bit blocks (which will then be encoded, bit by bit, using a stream cipher).<br>
<br>
Unfortunately, this is easily attacked. If the attacker knows three sequential plaintexts; $X_1, X_2, X_3$, (perhaps the document header is known, for example), then  simply by computing $S_1, S_2, S_3$, the constants in the LCG algorithm can be found:<br>
<br>
$A=(S_2-S_3)(S_1-S_2)^{-1}\mod{m}$<br>
$B=S_2-S_1(S_2-S_3)(S_1-S_2)^{-1}\mod{m}$

Further research [(Wikipedia)](https://en.wikipedia.org/wiki/Linear_congruential_generator) gives me more specifications: (paraphrased):

When $B≠0$, correctly chosen parameters allow a period equal to $m$, for all seed values. This will occur if and only if:

-  $m$ and the offset, $B$ are relatively prime,
-  $A-1$ is divisible by all prime factors of $m$,
-  $A-1$ is divisible by $4$ if $m$ is divisible by $4$.

Thus, to use the algorithm containing$\mod{2^{31}}$, given by the professor, I need to know all the prime factors of $2^{31}$ and then check $A-1$ is divisible by all. 

[A Wolfram Alpha tool](http://www.wolframalpha.com/widget/widgetPopup.jsp?p=v&id=8058bf500242562c0d031ff830ad094&title=Prime%20Factors&theme=&i0=2147483648&podSelect=) tells me there are $31$ prime factors in $2^{31}$, and all have a value of $2$. Since $A-1$ is even, this is divisible by all the prime factors of $2^{31}$.

### Encoding:
Now, let's try encoding some plaintext with a stream cipher that utilises LCG outputs.<br><br>
Let us assume we know the first three plaintext characters are 'c', 'a' and 't', represented by the ASCII codes $99, 97$, and $116$, respectively. We first need to represent each value within the 32-bit binary space defined by our$\mod{2^{31}}$

In [ ]:
# Convert decimal integers to 32-bit binary strings
X = [99, 97, 116]  # ASCII values of "c", "a" and "t".
Xbin = list(range(len(X)))

def get_long_binary(x):
    bin = get_binary(x)
    digits = (len(bin))
    for i in range (0, 32 - digits):
        bin = "0" + bin

    return bin

for i in range (0, len(X)):
    Xbin[i] = get_long_binary(X[i])

print("Thus, 'c', 'a', and 't', have the following 32-bit binary representations:")
print(Xbin)

Now, to encode these values, we simply need to XOR  each respective bit of binary plaintext value  with the respective bit of each respective LCG value. We can simply use the 32-bit binary representations of the PRNG values we generated above. To do this, we will also need to first make sure the LCG values are, indeed, 32 bits long:

In [ ]:
# Encoding Y1, Y2, Y3, with keystream S1, S2, S3 and plaintext X1, X2, X3 (ASCII codes in binary):

LCG_values = list(range(3))
stream = list(range(3))
decimal_encoded = list(range(3))

for i in range (0,3):
    LCG_values[i] = get_long_binary(keystream[i])
    stream[i] = encr_bin_str(LCG_values[i], Xbin[i])
    decimal_encoded[i] = int(stream[i], 2)
    
print("X values:        ", end=''); print(Xbin)
print("LCG values:      ", end=''); print(LCG_values)
print("cipher Y values: ", end=''); print(stream)
print("decimal representation of cipher stream (Y) values: ", end=''); print(decimal_encoded)

<a id='dvpa'></a>
### Decryption (via plaintext attack):
If we know the original plaintext values of the respective cipher text, we can now derive the values of the LCG constants, $A$ and $B$ (as if attacking this system without knowledge of their values, but with knowledge of the 3 sequential plaintext characters):

In [ ]:
S1 = keystream[0]; S2 = keystream[1]; S3 = keystream[2]

print("Previous encryption has given an encoding of 'cat' as S1, S2, and S3: %d, %d, %d:\n" % (S1, S2, S3))

def egcd(a, b):  # Find extended euclidean gcd, for use in modinv(), below.
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m): # Find modular inverse, if one exists (and throw exception if not)
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def find_A_B (S1, S2, S3):
    p1 = (S2 - S3) % (2**31)
    p2 = modinv(((S1 - S2) % (2**31)), 2**31)
    A = (p1 * p2) % (2**31)  # A = (S2 - S3)(S1 - S2)^-1 mod (2^31)
    B = (S2 - (S1 * A)) % 2**31  # B = S2 - S1(S2 - S3)(S1 - S2)^-1 mod (2^31)
    return A, B

A, B = find_A_B(S1, S2, S3)
print("A = %d\nB = %d\n" % (A, B))

S1 = S2; S2 = S3; S3 = keystream[3]
print("We can also solve using S2, S3, and S4: %d, %d, %d: \n" % (S1, S2, S3))

A, B = find_A_B(S1, S2, S3)
print("A = %d\nB = %d" % (A, B))

To derive $A$ and $B$ from the cipher stream mathematically, an attacker needs to know, at least, 3 sequential plaintext characters in the original message. To decrypt the message; the ciphertext, knowledge of how many bits represent each encrypted character (32, in this case), and knowledge of, or an ability to discover, the beginnings / ends of the 32-bit character binary values, (when beginning / ends unknown: brute-force; max 32 iterations, ave. 16 <-- hmm, wouldn't this give false positives, with no way to check if resultant PRNG uses same settings as used for encoding? N.b not very relevant where message reception is near flawless) and accurate interception of the complete / large enough portions of the cipher stream are also needed.<br>
<br>
So what is the plaintext _really_ needed for? To complete the decryption of the rest of a message (beyond the known plaintext) and/or any other message sent using the same PRNG (with the same key settings and adequate known plaintext, as described above). Once the values of $A$ and $B$ are discovered, then any subsequent LCG value can be discovered if the previous value is known. _Because_ our attacker knows some sequential plaintext, he or she can reverse engineer the respective cipherstream values to find the underlying LCG (keystream) values, thus facilitating the discovery of all subsequent keystream values, in turn facilitating decryption to plaintext by XOR of the respective cipher value with each respective keystream value. This also explains why knowledge of a file header plaintext, for example, might be ideal for decryption of more than one (possibly many) message(s). <br>
<br>
Time to put this all together in one big lump of code, methinks, including an actual message, of some length, for encryption and then decryption via plaintext attack:<br>

In [18]:
# Encrytion and decryption (via plaintext attack) of LCG-based stream cipher - by Simon Tharby, 2018
# STAND-ALONE PROGRAM - runs independently of other code execution

import numpy as np

# ------------------------------ Encryption: ------------------------------------------------

while True:
    message = input("Enter a message to encrypt: ")
    if len(message) > 2:
        break
    else:
        print("\nERROR! Message must be 3 or more characters long. Please try again.\n")

ascii = list(range(len(message))) # list to hold integer ascii values of message chars
for i in range (0, len(message)): # convert message chars to list of ascii value integers
    ascii[i] = ord(message[i])
    
x = 56789 # seed for LCG

keystream, binary = list(range(len(message))), list(range(len(message))) # lists for decimal & binary keystream values

get_binary = lambda x: format(x, 'b') # useful function for converting integer to binary string

for i in range (0, len(message)):  # LCG; calculate PRNs
    x = int((x * 1103515245 + 12345) % (2**31))
    keystream[i] = x
    binary[i] = get_binary(x)

Xbin = list(range(len(message)))  # list to hold 32-bit binary strings of message chars ascii values
Pbin = list(range(len(message)))  # list to hold 32-bit binary strings of LCG PRN values

def get_long_binary(a):  # Create 32-bit binary string from a decimal integer
    bin = get_binary(a)
    digits = (len(bin))
    for i in range (0, 32 - digits):
        bin = "0" + bin
    return bin

for i in range (0, len(message)):
    Xbin[i] = get_long_binary(ascii[i])
    Pbin[i] = get_long_binary(keystream[i]) # list of S1, S2, Sn+1 ... as binary 32-bit strings

# Create cipher stream; Y1, Y2, Y3, Yn+1 ... XOR binary PRN keystream with binary encoded values (ASCII codes):

cipher = list(range(len(message))) # list to hold 32-bit binary strings of cipher stream values
decimal_cipher = list(range(len(message)))  # list to hold decimal integer values of cipher stream values

def get_XOR(a, b):
    if a == b:
        result = 0
    else:
        result = 1
    return result

def encr_bin_str(data_in, key_stream):  # step through respective bits of two streams and XOR each pair of bits
    encoded = ""
    for i in range(len(data_in) * -1, 0):
        encoded = encoded + str(get_XOR(data_in[i], key_stream[i]))
    return encoded

for i in range (0, len(message)):  # step through each respective pair of 32-bit 'characters' from cipher and plaintext
    cipher[i] = encr_bin_str(Xbin[i], Pbin[i])
    decimal_cipher[i] = int(cipher[i], 2)

print("\nEncrypted message: ", end=''); print("".join('%s'%x for x in cipher))

# ------------------------------ Decryption (via plaintext attack): ---------------------------------

# 1. XOR bits of 3 known plaintext 32-bit binary encodings with the 3 respective cipher 32-bit binary encodings
#    to derive the respective LCG stream decimal integer values:

found_S = list(range(len(message)))   # list of derived 32-bit binary LCG keystream values S1, S2, Sn+1, ...
decimal_S = list(range(len(message))) # list of derived LCG keystream values as decimal integers
for i in range (0, 3):                # use known 3 plaintext values to derive 1st 3 values of keystream
    found_S[i] = encr_bin_str(Xbin[i], cipher[i])
    decimal_S[i] = int(found_S[i], 2)
    
S1 = decimal_S[0]; S2 = decimal_S[1]; S3 = decimal_S[2]

# 2. Derive A and B (LCG constants), from the 3 derived keystream values:

def egcd(a, b):  # Find extended euclidean gcd, for use in modinv, below
    count, floors = 0, []
    while b % a != 0:
        floors.append(b)
        a, b = b% a, a
        count += 1

    floors.append(b)
    g, s, t = abs(a), 0, 1
    for i in range (0, len(floors) - 1):
        s, t = t, s - ((floors[len(floors) - i - 2])  // (floors[len(floors) - i - 1])) * t
    return g, t, s

def modinv(a, m): # Find modular inverse, if one exists (and throw exception if not)
    g, s, t = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')

    return s % m

def find_A_B (S1, S2, S3):
    p1 = (S2 - S3) % (2**31)
    p2 = modinv(((S1 - S2) % (2**31)), 2**31)
    A = (p1 * p2) % (2**31)        # A = (S2 - S3)(S1 - S2)^-1
    B = (S2 - (S1 * A)) % (2**31)  # B = S2 - S1(S2 - S3)(S1 - S2)^-1
    return A, B

A, B = find_A_B(S1, S2, S3)

print("\nIf an attacker knows (or guesses) an LCG, using mod 2^31, was used to create a keystream constituted of 32-bit encoding,")
print("AND knows 3 original sequential plaintext values (and can locate them in cipher stream), then he or she can deduce the")
print("values of A and B in the LCG algorithm:    A = %d, B = %d" % (A, B))
print("\nNow, an attacker has all that is needed to decode the remaining cipher stream:\n")

# 3. Create remaining keystream values using LCG with newly discovered constants:

x = decimal_S[2]  # Use last (of 3) derived LCG PRNs as seed to create subequent outputs of LCG
for i in range (3, len(message)):
    x = int((x * A + B) % (2**31))
    decimal_S[i] = x
    found_S[i] = get_long_binary(x)
    
# 4. XOR all found keystream values with all 32-bit binary cipher text values:

binary_D = list(range(len(message)))  # list of decrypted cipher text as 32-bit binary strings
ascii_D = list(range(len(message)))   # list of decrypted cipher text ascii values as decimal integers

for i in range (0, len(message)):
    binary_D[i] = encr_bin_str(cipher[i], found_S[i])
    ascii_D[i] = int(binary_D[i], 2)
    
# 5. Convert decimal ascii values to characters:

decrypted = ""  # string variable to hold decrypted message text
for i in range (0, len(message)):
    decrypted = decrypted + chr(ascii_D[i])
    
print ("Decoded message: " + decrypted)

Enter a message to encrypt: decrypted with MY code! :-)

Encrypted message: 011010111011001110001001100011100010010001110011001101001011111000011011100010100110101100011011011111010111111010000010001000110101011000111000010110101100111101000000011001100100001111000111011010101011011011000111010100000100101001010111111100101110100001011001011001110110110000100110000100010001000101100100011100110011110000101011001100011110011100101100110111100010101011100000011110101000010000001010111110100001110101000011111100101100011101001100001110011101011010011100011001010111010111100110000010000011110110111010010000011100001101011111011001110000101011101011011010000010101110100010110010110001110000010011101000011010111000001011111010101101110100000010010001010101100110001000110000100110011011101100010000010111010100101001011101111001100011011101001010010011011101101001110010000101001000001100000010000110111000000000011101100001111011101001

If an attacker knows (or guesses) an LCG, using mod 2^31, 

### Plaintext attack (steps):    
1. Know or locate where the known plaintext begins in the cipher text stream.
2. XOR a 32-bit binary representation of the ascii value of each known plaintext character with the respective 32-bit cipher text block (minimum of 3 sequential plaintext characters must be known).
3. Use the 3 derived keystream values (from 2, above) to derive the LCG constants, A and B.
4. Use the third derived keystream value (from 2, above) as the seed for the LCG, and the known parameters of the LCG (from 3, above), to create the necessary subsequent sequence of keystream values.
5. XOR all 32-bit cipher text blocks with all respective derived/created keystream values to decrypt the entire message.

###### Further questions:
1. What if the beginning / end of the 32-bit binary 'character blocks' was not known (e.g. intercept of partial message)? Would brute force stepping through the stream, bit by bit, produce false positive values for A and B, sometimes, ever, always or never? (Easily testable, but perhaps only for an assumed case where it is known that some specific plaintext will  be in the message, but not exactly where. Because, to derive the LCG constants you must know exactly which cipher stream bits represent exactly each of the respective plaintext characters you know are in the message, this may not be simple to overcome. So, actually, it would only be possible to be sure the constants had been derived if the decrytion is successsful (and then detecting this may become the issue), and I have a suspicion any combination of values in $Z2^{31}$ will result in apparent constants, for the carefully chosen constants of the ANSII C rand() modelled here, but not necessarily, or at all, if the actual values of the bitstreams are offset. Need to check this.)
2. Similar to above, but what if the length (in bits) of the binary 'character blocks' was not known?
3. How many 'good' combinations of values of $A$ and $B$ can be found for, say, the$\mod{2^{31}}$ used here.
4. Right now, it seems obvious that no values prior to the PRNs, deduced from knowledge of plaintext, can be found for ciphertext that relates to plaintext found earlier in the message than the known plaintext, but is this true? Can we generate 'backwards' with a PRNG?
5. Given that $2^{31}$ has (only) $2$ as a divisor (other than itself and $1$, which all non-zero integers have), then how do we know the derivation of $A$ and $B$ (from 3 the LCG outputs) will always work. This can only be true if $S_1 - S_2$ never gives an even result, (since an even number $a$ cannot give a gcd of $1$ with the modulo $2^{31}$ (which is also even, hence the gcd must be 2 or more), hence no modular inverse of $a$ exists in such cases, and thus we could not derive $A$ and $B$), which can only happen if the LCG consistently produces alternating odd and even outputs. If the LCG used does this, then why? Given it facilitates an attack vector, is there something useful (or vital) to the LCG about this pattern? And/or is it the only way an LCG can give an illusion of randomness? e.g. EOOEEOOEE would also appear about as 'random', since neither seems very random in terms of 'odd/even-ness'? I still have lots of basic questions about LCGs (and probably PRNGs in general).

Thinking about it, a lot of q. 6 is answered by the Wikipedia notes I added to the description of an LCG, above. I am also interested in the 'period' being equal to the modulo $m$. Does this mean the LCG gives a repetitive cycle of the same values, with a period of $m$? Interesting, and easily testable (and if true, would probably better explain why LCGs are horrible for cryptography).

<a id='eea'></a>
## Extended Euclidean Algorithm and Modular Inverse:
I used some code that utilizes the Euclidean Extended Algorithm to find the modular inverse of an integer ring member (or to show there is none); `egcd()` and `modinv()` [[this tutorial]](https://discuss.codechef.com/questions/20842/a-tutorial-on-the-extended-euclids-algorithm). After a little investigation, (see [Slicing and Recursion (myBinder)](https://mybinder.org/v2/gh/jinjagit/JupyterNotebooks/master?filepath=slicing%20and%20recursion.ipynb)), I rewrote an iterative version of egcd(), (the original was recursive).<br>
1. The basic Euclidean Algorithm can be used to find the greatest common divisor (gcd) of two integers. It basically iterates through division of one number (the larger of the two) by the other, until a remainder of zero is found, with the gcd being the last non-zero remainder found before the zero remainder (except when the first remainder is zero, see note below). [Video explaining the Euclidean Algorithm [7 mins]](https://youtu.be/p5gn2hj51hs)
2. The Extended Euclidean Algorithm provides a more complex iterative (and can be recursive) method to find the values of $s$ and $t$ in Bezout's theorem, which states that for any two integers $a$ and $b$, there must be integers $t$ and $s$, such that $as + bt = \gcd{a, b}$. [Video explaining the Extended Euclidean Algorithm [15 mins]](https://youtu.be/6KmhCKxFWOs)
3. The abovementioned value of $t$, along with the $\gcd$, can be used to find the modular inverse of $a\mod{m}$, as used in the plaintext attack decryption of a stream cipher, above.

Time to code it, to get a better understanding of the code I used!<br>
<br>
Let's begin with the basic Euclidean Algorithm, to find the greatest common divisor ($\gcd$):

In [ ]:
# GCD calculator, using Euclidean Algorithm - by Simon Tharby, 2018
# STAND-ALONE PROGRAM - runs independently of other code execution

a = int(input("Enter 1st (non-zero) integer: "))
b = int(input("Enter 2nd (non-zero) integer: "))

def gcd(a, b):
    count = 0
    
    while b % a != 0:
        a, b = b% a, a
        count += 1
        
    return abs(a)

print("gcd: %d" % gcd(a, b))

Here is a rather beautiful and succinct recursive function, `egcd()`, taken from [this tutorial](https://discuss.codechef.com/questions/20842/a-tutorial-on-the-extended-euclids-algorithm) which uses the Extended Euclidean Algorithm to find $s$ and $t$ in Bezout's theorem, along with the $\gcd$:

In [21]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)

    g, y, x = egcd(b % a, a)  # <- this line contains the recursive call of the function itself
    return (g, x - (b // a) * y, y)
    
print(egcd(1180, 482))

(2, -29, 71)


Below is my iterative version of egcd(), which also finds $s$ and $t$ in Bezout's theorem, along with the $\gcd$.<br>
Also, see [Slicing and Recursion (myBinder)](https://mybinder.org/v2/gh/jinjagit/JupyterNotebooks/master?filepath=slicing%20and%20recursion.ipynb) for more details:

In [ ]:
# EGCD calculator (iterative), using Extended Euclidean Algorithm - by Simon Tharby, 2018

def egcd(a, b):
    count, floors = 0, []
    while b % a != 0:
        floors.append(b)
        a, b = b% a, a
        count += 1

    floors.append(b)
    g, s, t = abs(a), 0, 1
    for i in range (0, len(floors) - 1):
        s, t = t, s - ((floors[len(floors) - i - 2])  // (floors[len(floors) - i - 1])) * t
    return g, t, s

egcd(1180, 482)

Either of the above `egcd()` functions will provide the neccesary parameter values for the `modinv()` function, also taken from the aforementioned tutorial. `The modinv()` function is easy to follow, in the context of modular arithmetic, once the `egcd()` function is understood:

In [19]:
def modinv(a, m): # Find modular inverse, if one exists (and throw exception if not)
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')

    return x % m
    
modinv(4, 9)

7

Do LCGs repeat (and is the frequency related to the modulus)?<br>
If they repeat, is this a facet of LCGs with relatively 'well chosen' values for A and B, inevitable, or avoidable?<br>
<br>
Do all possible $S_n-S_{n+1}\mod{m}$ values of the C rand() emulation I am using have a modular inverse?<br>

<br>
***
<br>
Notes on Lecture 4, [Introduction to Cryptography](https://www.youtube.com/channel/UC1usFRN4LCMcfIV7UjHNuQg/videos),by Christof Paar, and Chapter 2 (cont.), [Understanding Crpytography](https://www.amazon.com/Understanding-Cryptography-Christof-Paar-ebook/dp/B00HWUO98A), by Paar & Pletzl:<br>

<br>
***
### References:
Lectures 3 and 4, [Introduction to Cryptography](https://www.youtube.com/channel/UC1usFRN4LCMcfIV7UjHNuQg/videos),by Christof Paar
[Understanding Crpytography](https://www.amazon.com/Understanding-Cryptography-Christof-Paar-ebook/dp/B00HWUO98A), by Paar & Pletzl:<br>
<bnr>
Extended Euclidian Algo: http://anh.cs.luc.edu/331/notes/xgcd.pdf / https://discuss.codechef.com/questions/20842/a-tutorial-on-the-extended-euclids-algorithm<br>
Python Extended Euclidean Algorithm and Modular Inverse: https://discuss.codechef.com/questions/20842/a-tutorial-on-the-extended-euclids-algorithm

Document unfinished. In progress...

<center><a href='#top'>Back to top of document</a><br></center>